In [1]:
import csv
import email_validator
import os
import tkinter as tk



In [2]:
class App(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)

        self.file_path = ""

        # Create a label for the file name
        self.file_name_label = tk.Label(self, text="File Name: ")

        # Create a text entry for the file name
        self.file_name_entry = tk.Entry(self)

        # Create a button to open the file
        self.open_file_button = tk.Button(self, text="Open File", command=self.open_file)

        # Create a button to clean the file
        self.clean_file_button = tk.Button(self, text="Clean File", command=self.clean_file)

        # Layout the widgets
        self.file_name_label.pack()
        self.file_name_entry.pack()
        self.open_file_button.pack()
        self.clean_file_button.pack()

    def open_file(self):
        # Get the file path from the user
        self.file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv"), ("Excel files", "*.xlsx")])

        # Check if the file exists
        if not self.file_path:
            return

        # Clean the file
        self.clean_file()

    def clean_file(self):
        # Get the file path
        file_path = self.file_path

        # Open the file
        with open(file_path, "r") as f:
            reader = csv.reader(f)

            # Get the column names
            column_names = next(reader)

            # Create a list of dictionaries to store the data
            data = []
            for row in reader:
                data.append({
                    column_name: row[column_name] for column_name in column_names
                })

            # Clean the phone numbers
            for phone_number in data:
                if phone_number["Main Phone"].startswith("020") or phone_number["Main Phone"].startswith("20"):
                    phone_number["Main Phone"] = ""

            # Save the cleaned data to a new file
            directory = os.path.dirname(file_path)
            file_name = os.path.basename(file_path)
            new_file_path = os.path.join(directory, "clean_" + file_name)
            with open(new_file_path, "w") as f:
                writer = csv.writer(f)
                writer.writerow(column_names)
                writer.writerows(data)

        print("The file has been cleaned successfully.")



if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()